**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
import skvideo.io
import cv2
import json
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D,MaxPooling2D,Activation,AveragePooling2D,Reshape,BatchNormalization,Dropout,Flatten,Convolution2D
from keras.utils import np_utils

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer. *texte en italique*

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4,min_eps = 0.01):
        self.epsilon = epsilon
        self.n_action = n_action
        self.min_eps = min_eps
    
    def set_epsilon(self,e):
        if(e > self.min_eps):
          self.epsilon = e
        

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
#__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act takes as input the current state and a boolean value (indicating whether we're considering the train or the test procedure ).

- If we call act in the training procedure, we have 2 possibilities:  we can either act through learned_act (i.e. according to the policy of the agent) or take a random action.

- If we call act in the test procedure: we act always according to the learned policy.

epsilon is a variable $\in \left[ 0, 1\right]$ which accounts for the probability of agent to choose a random action. 
This variable is essential since it permits to tune the **exploration ** - ** exploitation**  trade-off: at each time we can either explore or follow our policy; for epsilon $\sim 0$ the agent follows always the policy at tends to learn nothing (since it does not explore the environment!), while forepsilon $\sim 1$ the agent explores the environment, not considering the policy (and thus its actions does not take into account the past).

***
### The Game

In [0]:
#The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

#```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(e)

        # Update stats
        score += win-lose

        #print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})".format(e, epoch, loss, win, lose, win-lose))

        agent.save()


KeyboardInterrupt: ignored

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=13, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board<0,2] = 256
        b[self.x,self.y,:]= 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        
        # set the current position
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # take the old value
        reward = self.board[self.x, self.y]
        # in the bord there's nothing, in the correspondent position
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=50
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

#__Question 2__ :Explain the use of the arrays ```position``` and ```board```.

Both `position` and board are two matrixes of dimension (grid_size+4)  $\times$ (grid_size+4), the term +4 accounting for the the grid boards.

1.   `position`: it tracks the postion during the game. it's  inizialized to 0. Takes 1 in the position $\left( x,y \right) $ as value if the agent has passed in the $\left( x,y \right) $  position. Bords entries, i.e., entries whose indexes are $\left( [0:2,:], [:,0:2], [-2:,:], [:,-2:]\right)$ equal  - 1 and cannot be visited.
2.   `board`: it acts like a reward field.  Its $\left( i, j \right)$ element are initialized to the value $bonus_{\left( i, j \right)}$ + $1\!\!1_{bonus_{\left( i, j \right)} > 0}$ $\cdot$ $malus_{\left( i, j \right)}$, where ${\{bonus \}}_{  i, j  = 1 }^{grid size}$  = $\frac{B\left(temperature)\right)}{2}$ and ${\{malus\}}_{  i, j  = 1 }^{grid size}$  = $- B\left(temperature\right)$, 
$B\left(temperature\right)$ indicating a r.v. of parameter `temperature`.

Those two matrix defines the state of the game.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, 4, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action, False)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    #if e % 10 == 0:
        #env.draw(e)

    # Update stats
    #score += win-lose

    #print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          #.format(e, epoch, loss, win, lose, win-lose))
    #agent.save()
        ##### FILL IN HERE
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 3.5/9.0. Average score (-5.5)
Win/lose count 4.0/9.0. Average score (-5.25)
Win/lose count 1.5/1.0. Average score (-3.3333333333333335)
Win/lose count 2.5/1.0. Average score (-2.125)
Win/lose count 4.5/3.0. Average score (-1.4)
Win/lose count 2.5/2.0. Average score (-1.0833333333333333)
Win/lose count 4.5/3.0. Average score (-0.7142857142857143)
Win/lose count 2.5/5.0. Average score (-0.9375)
Win/lose count 3.5/7.0. Average score (-1.2222222222222223)
Win/lose count 3.5/8.0. Average score (-1.55)
Final score: -1.55


***
## DQN

Let us assume here that $T=\infty$.

***
#__Question 5__: Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

##Proof

\begin{equation*}

\begin{split}
Q^{\pi}(s,a) &=E[  \sum_{k = 0}^{\infty}{{\gamma}^{k}r_{k+1}\mid s_{0} = s, a_{0} = a]} \\
 &=E [ r(s,a) + \gamma \sum_{k = 0}^{\infty}{{\gamma}^{k}r_{k+2}\mid s_{0} = s, a_{0} = a]} \\
 &=E [ r(s,a) + \gamma  E[ \sum_{k = 0}^{\infty}{{\gamma}^{k}r_{k+2} \mid s_{1} = s', a_{1} = a'] \mid s_{0} = s, a_{0} = a]} \\
 &=E_{\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{split}

\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

## Proof

Let be $\pi^*$ the optmila policy. Then $q^{*}$ must satisfy :
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)] +\gamma \sum_{s' \in S} p(s'\mid s,a) v^{*}(s').
\end{equation*}
since $v^{*}(s) = {\max}_{a \in A} Q^{*}(s,a)$, we conclude.


# Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

We want our NN to learn the optimal Q function by penalizing the difference between  the predicted function and the structure of the optimal one. Notice that we don't have acces to the true Q function value and therefore this is not a supervised problem. The only feedbacks the algorithm receives from the environment  are the next state and the reward. In this case the target is $ r+\gamma\max_{a'}Q(s',a',\theta)$ and we look for $Q(s,a,\theta)$ which has the property to obey to the bellman optimality equation.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if(len(self.memory) < self.max_memory):
            self.memory.append(m)
            self.max_memory += 1
        else:
            self.memory[np.random.randint(len(self.max_memory))] = m

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            
            action = agent.act(state)
                        
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
import random
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=3):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.01
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size,5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
                
        for i in range(self.batch_size):
            
            episode = self.memory.random_access()
            
            if episode[4]:
                input_states[i] = episode[0]
                target_q[i,:] = episode[3]
            else:
                input_states[i] = episode[0]
                target_q[i,int(episode[2])] = episode[3] + self.discount*np.amax(self.model.predict(episode[1].reshape((1,5,5,self.n_state)))) 
                
        target_q = np.clip(target_q, -8, 8)
        l = self.model.train_on_batch(input_states, target_q)

        return l


    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
                
        model = keras.Sequential()
        model.add(Dense(200,activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Dropout(0.25))
        model.add(Dense(1000,activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(50,activation='relu'))
        model.add(Flatten())
        model.add(Dense(4,activation='softmax'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()
    
    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

In [0]:
epochs_train = 60
T = 50
env = Environment(grid_size=size, max_time=T, temperature=0.2)
agent = DQN_FC(size, lr=.1, epsilon = 0.45, memory_size=2000, batch_size = 32, n_state = 2)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/060 | Loss 0.0852 | Win/lose count 3.5/3.0 (0.5)
Epoch 001/060 | Loss 0.1322 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/060 | Loss 0.0853 | Win/lose count 1.5/3.0 (-1.5)
Epoch 003/060 | Loss 0.0734 | Win/lose count 1.0/4.0 (-3.0)
Epoch 004/060 | Loss 0.1085 | Win/lose count 2.5/7.0 (-4.5)
Epoch 005/060 | Loss 0.0622 | Win/lose count 0.5/0 (0.5)
Epoch 006/060 | Loss 0.0970 | Win/lose count 1.0/4.0 (-3.0)
Epoch 007/060 | Loss 0.0622 | Win/lose count 2.0/5.0 (-3.0)
Epoch 008/060 | Loss 0.1091 | Win/lose count 0.5/7.0 (-6.5)
Epoch 009/060 | Loss 0.0970 | Win/lose count 1.5/3.0 (-1.5)
Epoch 010/060 | Loss 0.0623 | Win/lose count 1.0/2.0 (-1.0)
Epoch 011/060 | Loss 0.1086 | Win/lose count 2.5/2.0 (0.5)
Epoch 012/060 | Loss 0.1206 | Win/lose count 1.5/2.0 (-0.5)
Epoch 013/060 | Loss 0.0621 | Win/lose count 0/3.0 (-3.0)
Epoch 014/060 | Loss 0.0856 | Win/lose count 3.0/2.0 (1.0)
Epoch 015/060 | Loss 0.0853 | Win/lose count 4.5/1.0 (3.5)
Epoch 016/060 | Loss 0.0737 | Win/lose count 0.5/

In [0]:
HTML(display_videos('fc_train0.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential()
        model.add(Convolution2D(32, (2, 2), input_shape=(5,5,self.n_state)))
        model.add(Convolution2D(64, (2, 2)))
        model.add(Flatten())
        model.add(Dense(1024)) 
        model.add(Activation('relu'))
        model.add(Dense(4))
        model.add(Activation('softmax'))        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()

        
    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

In [87]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32,n_state = 2)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 3, 3, 64)          8256      
_________________________________________________________________
flatten_17 (Flatten)         (None, 576)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 1024)              590848    
_________________________________________________________________
activation_21 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 4)                 4100      
_________________________________________________________________
activation_22 (Activation)   (None, 4)                 0         
Total para

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [88]:
env = Environment(grid_size=size, max_time=T,temperature=0.2)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state = 2)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state = 2)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 3, 3, 64)          8256      
_________________________________________________________________
flatten_18 (Flatten)         (None, 576)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 1024)              590848    
_________________________________________________________________
activation_23 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 4)                 4100      
_________________________________________________________________
activation_24 (Activation)   (None, 4)                 0         
Total para

## Test of the FC


In [89]:
test(agent_fc,env,epochs_test,prefix='fc_test')

Win/lose count 0.5/4.0. Average score (-3.5)
Win/lose count 2.0/3.0. Average score (-2.25)
Win/lose count 0.5/0. Average score (-1.3333333333333333)
Win/lose count 0/1.0. Average score (-1.25)
Win/lose count 0/1.0. Average score (-1.2)
Win/lose count 1.5/0. Average score (-0.75)
Win/lose count 1.0/1.0. Average score (-0.6428571428571429)
Win/lose count 0/0. Average score (-0.5625)
Win/lose count 1.5/0. Average score (-0.3333333333333333)
Win/lose count 0/1.0. Average score (-0.4)
Final score: -0.4


In [90]:
 HTML(display_videos('fc_test0.mp4'))

## Test of the CNN

In [91]:
test(agent_cnn,env,epochs_test,prefix='cnn_test')

Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0.5/0. Average score (0.75)
Win/lose count 2.0/0. Average score (1.1666666666666667)
Win/lose count 1.0/0. Average score (1.125)
Win/lose count 1.0/0. Average score (1.1)
Win/lose count 3.0/0. Average score (1.4166666666666667)
Win/lose count 0.5/0. Average score (1.2857142857142858)
Win/lose count 0.5/0. Average score (1.1875)
Win/lose count 2.5/0. Average score (1.3333333333333333)
Win/lose count 0/0. Average score (1.2)
Final score: 1.2


In [92]:
HTML(display_videos('cnn_test0.mp4'))

# Comment
Three temperature values has been considered: T =0.2,  0.4, 0.8

[T = 0.2]:  since the beginning the agent does not explore the map and often turns around to its initial position. In the testing phase: with the FF NN the agent starts oscillating since the very beginning, with the CNN one the agent explores few cells before being stuck visiting the sames cells.

[T = 0.4] :   the agent's behaviour is very similar to the previous case. CNN improves this aspect, but still the agent visits many times the same cells.

[T = 0.8] :  in the training phase,  the agent explores new cells more frequently than the previous case. Nevertheless, in the testing phase the agent starts oscillating after few steps. 


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    dc = 0.99
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            
            action = agent.act(state)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = round(win + reward,2)
                
            if reward < 0:
                lose = round(lose - reward,2)

            # Apply the reinforcement strategy
            loss = round(agent.reinforce(prev_state, state,  action, reward, game_over),2)
            
        # Update the epsilon when the game is over
        agent.set_epsilon(agent.epsilon*dc)          

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += round(win-lose,2)
        

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=13, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board<0,2] = 256
        b[self.x,self.y,:]= 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = - self.malus_position[self.x, self.y] 
        self.malus_position[self.x, self.y] += 0.55 
        reward += self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
          
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0
        
        # Reset the board
        self.board = bonus + malus
        
        # Reset the position
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        # Reset the malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))


        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state
    
        ## use those samples of code:
        #In train explore:
        #state, reward, game_over = env.act(action, train=True)  
        
        #reward = reward + self.board[self.x, self.y]
        # 3 "feature" states instead of 2
        #state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
        #                                self.board.reshape(self.grid_size, self.grid_size,1),
        #                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [100]:
# Training
epochs_train = 100 
T = 120
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.4)
print(env.grid_size)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=20000,batch_size = 32, n_state = 3)


17
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 4, 4, 32)          416       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 3, 3, 64)          8256      
_________________________________________________________________
flatten_22 (Flatten)         (None, 576)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 1024)              590848    
_________________________________________________________________
activation_29 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_58 (Dense)             (None, 4)                 4100      
_________________________________________________________________
activation_30 (Activation)   (None, 4)                 0         
Total p

In [101]:
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore0.mp4'))

Epoch 000/100 | Loss 0.6400 | Win/lose count 10.0/100.3 (-90.3)
Epoch 001/100 | Loss 0.4600 | Win/lose count 9.5/109.45 (-99.95)
Epoch 002/100 | Loss 0.4700 | Win/lose count 11.0/88.1 (-77.1)
Epoch 003/100 | Loss 0.4300 | Win/lose count 9.0/103.1 (-94.1)
Epoch 004/100 | Loss 0.3900 | Win/lose count 7.0/96.25 (-89.25)
Epoch 005/100 | Loss 0.5500 | Win/lose count 9.0/97.9 (-88.9)
Epoch 006/100 | Loss 0.6200 | Win/lose count 8.0/116.1 (-108.1)
Epoch 007/100 | Loss 0.3500 | Win/lose count 8.0/95.0 (-87.0)
Epoch 008/100 | Loss 0.6400 | Win/lose count 11.5/87.35 (-75.85)
Epoch 009/100 | Loss 0.4300 | Win/lose count 14.0/67.75 (-53.75)
Epoch 010/100 | Loss 0.5100 | Win/lose count 12.0/94.1 (-82.1)
Epoch 011/100 | Loss 0.5300 | Win/lose count 10.5/110.95 (-100.45)
Epoch 012/100 | Loss 0.3100 | Win/lose count 15.5/66.35 (-50.849999999999994)
Epoch 013/100 | Loss 0.4500 | Win/lose count 2.5/163.85 (-161.35)
Epoch 014/100 | Loss 0.7200 | Win/lose count 9.5/100.35 (-90.85)
Epoch 015/100 | Loss 0.3

In [103]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 25.5/4.0. Average score (21.5)
Win/lose count 13.5/6.0. Average score (14.5)
Win/lose count 26.0/4.0. Average score (17.0)
Win/lose count 31.5/2.0. Average score (20.125)
Win/lose count 33.5/1.0. Average score (22.6)
Win/lose count 27.5/4.0. Average score (22.75)
Win/lose count 22.5/5.0. Average score (22.0)
Win/lose count 25.5/4.0. Average score (21.9375)
Win/lose count 16.5/7.0. Average score (20.555555555555557)
Win/lose count 28.0/6.0. Average score (20.7)
Final score: 20.7


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***